<a href="https://colab.research.google.com/github/USCbiostats/PM520/blob/main/Lab_8_Variational_Inference_PtI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running Up That Hill, or: Intro to Variational (Bayesian) Inference
TBD